In [ ]:
! git clone https://github.com/maxstclair/project_chd

Cloning into 'project_chd'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 13 (delta 5), reused 4 (delta 4), pack-reused 6
Receiving objects: 100% (13/13), 635.14 KiB | 5.29 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn import linear_model

# Import dataset
df = pd.read_csv('./project_chd/fhs_train.csv')
df = df.drop('Unnamed: 0', axis=1)
df_test = pd.read_csv('./project_chd/fhs_test.csv')
df_test = df_test.drop('Unnamed: 0', axis=1)

# handle NAs by dropping
df = df.dropna()
df_test = df_test.dropna()
df.head()

,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,58,1.0,0,0.0,0.0,0,0,0,220.0,143.0,104.0,29.85,75,87.0,1
1,0,40,1.0,1,15.0,0.0,0,0,0,199.0,122.0,82.0,22.16,85,77.0,0
3,1,38,2.0,1,43.0,0.0,0,1,0,170.0,130.0,94.0,23.90,110,75.0,0
4,0,43,1.0,0,0.0,0.0,0,0,0,202.0,124.0,92.0,21.26,75,74.0,0
6,0,54,1.0,0,0.0,0.0,0,1,0,237.0,171.5,105.5,34.25,91,104.0,0


In [ ]:
vars = ['age', 'totChol',	'sysBP',	'diaBP',	'BMI',	'heartRate',	'glucose'] # quantitative vars we want to log and interact with gender
for i in vars:
  df[i+'_log'] = np.log(df[i]) # take the log form of variables
  df[i+'_sex'] = df[i]*df['sex'] # create an interaction term with sex

# same procedure for test set
for i in vars:
  df_test[i+'_log'] = np.log(df_test[i])
  df_test[i+'_sex'] = df_test[i]*df_test['sex']
df.head()

,sex,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,...,sysBP_log,sysBP_sex,diaBP_log,diaBP_sex,BMI_log,BMI_sex,heartRate_log,heartRate_sex,glucose_log,glucose_sex
1,0,51,3.0,0,0.0,0.0,0,0,0,264.0,...,4.905275,0.0,4.418841,0.0,3.283914,0.00,4.094345,0.0,4.304065,0.0
4,1,58,2.0,1,20.0,0.0,0,0,0,207.0,...,4.700480,110.0,4.382027,80.0,3.159126,23.55,4.356709,78.0,4.356709,78.0
5,1,53,1.0,0,0.0,0.0,0,1,0,202.0,...,4.945207,140.5,4.499810,90.0,3.251149,25.82,4.442651,85.0,4.304065,74.0
6,1,41,2.0,1,5.0,0.0,0,0,0,218.0,...,4.863681,129.5,4.532599,93.0,3.325036,27.80,4.060443,58.0,4.418841,83.0
7,1,50,4.0,1,35.0,0.0,0,0,0,236.0,...,4.624973,102.0,4.234107,69.0,3.090133,21.98,4.248495,70.0,4.290459,73.0


In [ ]:
import statsmodels.api as sm

X_train = df.drop('TenYearCHD',axis=1)
y_train = df['TenYearCHD']
# Add a constant to the features
X_train = sm.add_constant(X_train)

# Fit the model
model = sm.OLS(y_train, X_train).fit()

# Print summary statistics including p-values
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             TenYearCHD   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     12.34
Date:                Tue, 23 Apr 2024   Prob (F-statistic):           2.51e-54
Time:                        02:05:25   Log-Likelihood:                -931.24
No. Observations:                2744   AIC:                             1922.
Df Residuals:                    2714   BIC:                             2100.
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               8.6241      2.611     

In [ ]:
from sklearn.linear_model import LinearRegression

X = df.drop('TenYearCHD',axis=1)
y = df['TenYearCHD']
X_test = df_test.drop('TenYearCHD',axis=1)
y_test = df_test['TenYearCHD']

# Fit model
model = LinearRegression()
lm = model.fit(X,y)

# Model predictions
y_hat = lm.predict(X_test)

SSE = np.sum((y_test - y_hat)**2)
TSS = np.sum((y_test - np.mean(y))**2)
rsq = 1 - SSE/TSS
print(rsq)

0.10198401086877873
